# <center> Twitter API </center>

In [ ]:
import sys
import socket
import json
from datetime import datetime
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
HOST = '17.18.0.2'  # IP address of spark-master
PORT = 9009  
s.bind((HOST, PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()

class StdOutListener(StreamListener):
    def on_data(self, data):
        #print(data)        
        # extract tweet contents only
        full_tweet = json.loads(data)
        tweet_text = full_tweet.get('text')
        print(tweet_text)
        conn.sendall(str.encode(tweet_text + '\n'))
        return True
    def on_error(self, status):
        print (status)
        
# read cert (not on github)
keyFile = open('/tmp/twitter','r')
CONSUMER_KEY = keyFile.readline().rstrip()
CONSUMER_SECRET = keyFile.readline().rstrip()
ACCESS_TOKEN_KEY = keyFile.readline().rstrip()
ACCESS_TOKEN_SECRET = keyFile.readline().rstrip()

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

while True:
    try:
        stream = Stream(auth, StdOutListener())
        # http://bboxfinder.com/#39.806426,-75.465088,40.143190,-74.794922
        stream.filter(locations=[-75.465088,39.806426,-74.794922,40.143190])
    except IncompleteRead:
        print ("Lets try it again")
        continue
    except KeyboardInterrupt:
        s.close()
        stream.disconnect()
        break

Waiting for TCP connection...
